In [1]:
#coding:utf-8
%reload_ext autoreload 
%autoreload 2
import os
import sys

project_basedir = '..'
sys.path.append(project_basedir)
from cchess_zero import cbf
from cchess_zero import board_visualizer
from matplotlib import pyplot as plt
import os
from cchess_zero.gameboard import *
from net import resnet
import common
from gameplays import gameplay,players
from gameplays.analysis import get_km_json
import json
%matplotlib inline

/usr/local/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [34]:

fullpath = '../data/distributed/2018-07-25_10-33-13_166_mcts-mcts_net-net_w.cbf'
moves = cbf.cbf2move(fullpath)
fname = fullpath.split('/')[-1]
print(fname)
state = gameplay.GameState()

2018-07-25_10-33-13_166_mcts-mcts_net-net_w.cbf


In [40]:
from PIL import Image
import imageio
imgs_for_gif = []
state = gameplay.GameState()
statestr = 'RNBAKABNR/9/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c5c1/9/rnbakabnr'
for i in moves[:-1]: 
    statestr = GameBoard.sim_do_action(i,statestr)
    imgs_for_gif.append(board_visualizer.get_board_img(statestr))
    state.do_move(i)
imgs_for_gif = imgs_for_gif[-10:]
imageio.mimsave('../data/gif/{}-.gif'.format(fname[:-4]), imgs_for_gif,duration=0.5)

In [41]:
i,len(moves)

('a9b9', 68)

In [42]:
def display_gif(fn):
    from IPython import display
    return display.HTML('<img src="{}">'.format(fn))

In [38]:
display_gif('../data/gif/{}-.gif'.format(fname[:-4]))

# ANALYSIS

In [19]:
network = resnet.get_model(os.path.join(project_basedir,
    '../data/prepare_weight/2018-07-21_21-13-25'[3:]),
    common.board.create_uci_labels(),GPU_CORE=[None],FILTERS=128,NUM_RES_LAYERS=7)

0
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
inside res net [None, 10, 9, 128]
INFO:tensorflow:Restoring parameters from ../data/prepare_weight/2018-07-21_21-13-25


In [43]:
network_player = players.NetworkPlayer(state.currentplayer,network,n_playout=400)

In [44]:
network_player.make_move(state=state,actual_move=False)

('e8d9', -0.9991512879729275)

In [47]:
childs = network_player.mcts_policy._root._children
for move in childs:
    val = childs[move]
    print(move,val._n_visits,val._P,val._Q)

b5a7 2 2.1147218e-05 -1.0
b5c7 1 7.377319e-05 -1.0
b5d6 1 0.00023399299 -1.0
b5a3 2 0.0010998154 -1.0
b5d4 2 0.0009843453 -1.0
b5c3 1 0.00020102841 -1.0
e9d9 24 0.01026618 -1.0
i8f8 18 0.00783159 -1.0
i8g8 13 0.0057326653 -1.0
i8h8 2 0.0012406321 -1.0
i8i9 1 0.0001555783 -1.0
i7g8 4 0.0017658012 -1.0
i7g6 3 0.0013134509 -1.0
i7h5 1 0.00054636406 -1.0
e8d9 274 0.8921723 -1.0
e8f7 1 4.440104e-05 -1.0
e8d7 4 0.0018076225 -1.0
e7c9 43 0.018552644 -1.0
e7g5 1 4.3327265e-05 -1.0
e7c5 1 0.00014601088 -1.0


In [28]:
datajson = get_km_json(network_player.mcts_policy,state.statestr,depth=40)

In [29]:
with open('../data/mstc_vis/sample.km','w') as whdl:
    json.dump(datajson,whdl)